In [2]:
const { z } = require('zod');
const { tool } = require('@langchain/core/tools');

const { ChatBedrockConverse } = require("@langchain/aws");
const { HumanMessage, AIMessage, SystemMessage, BaseMessage } = require("@langchain/core/messages");
const { createReactAgent } = require("@langchain/langgraph/prebuilt");
const { Calculator } = require("@langchain/community/tools/calculator");
const { EventEmitter } = require("events");

process.env.AWS_DEFAULT_REGION='us-east-1'

let llm
let tools
let agent
let main
let myNewToolSchema
let myNewToolDefinitoin

'us-east-1'

In [5]:
//Define a ZOD object with the tool argument schema
myNewToolSchema = z.object({
    title: z.string(),
    description: z.string(),
    buttonTextBeforeClick: z.string(),
    buttonTextAfterClick: z.string(),
})

myNewToolDefinition = tool(
    async (userInputToolArgs) => {
        return {
            success: true
        }
    },
    {
        name: "userInputTool",
        description: `
Use this tool to send emails or add items to a work management system.
The messages should never request information.
They should only inform someone besides the user about an action they should take (including to review an item from the chat).
`,
        schema: myNewToolSchema,
    }
);

DynamicStructuredTool {
  lc_serializable: false,
  lc_kwargs: {
    name: 'userInputTool',
    description: '\n' +
      'Use this tool to send emails or add items to a work management system.\n' +
      'The messages should never request information.\n' +
      'They should only inform someone besides the user about an action they should take (including to review an item from the chat).\n',
    schema: ZodObject {
      spa: [Function: bound safeParseAsync] AsyncFunction,
      _def: [Object],
      parse: [Function: bound parse],
      safeParse: [Function: bound safeParse],
      parseAsync: [Function: bound parseAsync] AsyncFunction,
      safeParseAsync: [Function: bound safeParseAsync] AsyncFunction,
      refine: [Function: bound refine],
      refinement: [Function: bound refinement],
      superRefine: [Function: bound superRefine],
      optional: [Function: bound optional],
      nullable: [Function: bound nullable],
      nullish: [Function: bound nullish],
      array: [F

In [6]:
// Initialize Bedrock LLM
llm = new ChatBedrockConverse({
    model: "us.anthropic.claude-3-5-haiku-20241022-v1:0"
});

// Define available tools
tools = [
    myNewToolDefinition,
    
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools,
});

CompiledStateGraph {
  lc_serializable: false,
  lc_kwargs: {
    checkpointer: undefined,
    interruptAfter: undefined,
    interruptBefore: undefined,
    autoValidate: false,
    nodes: {
      __start__: [PregelNode],
      agent: [PregelNode],
      tools: [PregelNode]
    },
    channels: {
      messages: [BinaryOperatorAggregate],
      structuredResponse: [LastValue],
      __start__: [EphemeralValue],
      agent: [EphemeralValue],
      tools: [EphemeralValue],
      'branch:__start__:__self__:agent': [EphemeralValue],
      'branch:__start__:__self__:tools': [EphemeralValue],
      'branch:agent:__self__:agent': [EphemeralValue],
      'branch:agent:__self__:tools': [EphemeralValue],
      'branch:tools:__self__:agent': [EphemeralValue],
      'branch:tools:__self__:tools': [EphemeralValue],
      '__start__:agent': [EphemeralValue],
      'branch:agent:condition:tools': [EphemeralValue]
    },
    inputChannels: '__start__',
    outputChannels: [ 'messages', 'structuredRe

In [9]:
main = async () => {
    const agentFinalState = await agent.invoke(
      { messages: [new HumanMessage("I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico. Send him an email.")] }
    );
    console.log(agentFinalState)
}

main()

Promise { <pending> }

{
  messages: [
    HumanMessage {
      "id": "a2f1e7a4-1430-4b4d-a601-5f7a46459848",
      "content": "I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico. Send him an email.",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "id": "8cda9a0e-5e25-434b-b789-b7982a7a2ca1",
      "content": [
        {
          "type": "text",
          "text": "I'll help you send an email to Jeff about purchasing a new downhole pump for the Fruitland Coal well in New Mexico. I'll use the userInputTool to draft and send the email."
        }
      ],
      "additional_kwargs": {},
      "response_metadata": {
        "$metadata": {
          "httpStatusCode": 200,
          "requestId": "8cda9a0e-5e25-434b-b789-b7982a7a2ca1",
          "attempts": 1,
          "totalRetryDelay": 0
        },
        "metrics": {
          "latencyMs": 3795
        },
        "stopReason": "tool_use",
        "usage": {
          "cacheReadInputTok

# 